# Сбор данных

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import requests
import lxml.html
from pprint import pprint

### Обработка html-страниц
### hh.ru

In [2]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.0; rv:14.0) Gecko/20100101 Firefox/14.0.1',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'ru-ru,ru;q=0.8,en-us;q=0.5,en;q=0.3',
    'Accept-Encoding':'gzip, deflate',
    'Connection':'keep-alive',
    'DNT':'1'
}

vacancies = ['python-разработчик', 'devops']
df = pd.DataFrame(columns=['vacancy', 'desription'])
for vacancy in vacancies:
    url = 'https://hh.ru/search/vacancy?clusters=true&text=' + vacancy + '&enable_snippets=true&page=0&search_field=name&no_magic=true'
    request = requests.get(url, headers=headers)
    soup = BeautifulSoup(request.text, 'lxml')
    css_selector_vacancy_urls = '.HH-LinkModifier'
    vacancy_urls = list(map(lambda x: x['href'], soup.select(css_selector_vacancy_urls)))
    for url in vacancy_urls[:2]:
        request = requests.get(url, headers=headers)
        soup = BeautifulSoup(request.text, 'lxml')
        css_selector_desription = '.g-user-content'
        vacancy_text = soup.select(css_selector_desription)[0].text
        df.loc[len(df)] = [vacancy, vacancy_text]
df

,vacancy,desription
0,python-разработчик,Мы ищем ведущего разработчика для продуктивн...
1,python-разработчик,Обязанности: Разработка проектов на Django/Fl...
2,devops,Мы ищем Devops Engineer для участия в развитии...
3,devops,Требуемый опыт работы: от 2 лет Полная занятос...


### imdb.com

In [3]:
films = ['Balto', 'Hellboy', 'Fargo', 'Aliens']
df = pd.DataFrame(columns=['title', 'overview'])
for film in films:
    url = 'https://www.imdb.com/find?ref_=nv_sr_fn&q=' + film + '&s=all'
    request = requests.get(url, headers=headers)
    soup = BeautifulSoup(request.text, 'lxml')
    css_selector_film_url = '.result_text a'
    film_url = 'https://www.imdb.com' + soup.select(css_selector_film_url)[0]['href']
    request = requests.get(film_url, headers=headers)
    soup = BeautifulSoup(request.text, 'lxml')
    css_selector_overview = '.summary_text'
    overview_text = soup.select(css_selector_overview)[0].text.strip()
    df.loc[len(df)] = [film, overview_text]
df

,title,overview
0,Balto,An outcast half-wolf risks his life to prevent...
1,Hellboy,"Based on the graphic novels by Mike Mignola, H..."
2,Fargo,"Various chronicles of deception, intrigue and ..."
3,Aliens,Ellen Ripley is rescued by a deep salvage team...


## Обработка json
### api.hh.ru
https://github.com/hhru/api

In [6]:
url = 'https://api.hh.ru/vacancies?industry=7&per_page=1&page=0'
request = requests.get(url)
pprint(request.json())

{'alternate_url': 'https://hh.ru/search/vacancy?enable_snippets=true&industry=7&items_on_page=1',
 'arguments': None,
 'clusters': None,
 'found': 55395,
 'items': [{'address': None,
            'alternate_url': 'https://hh.ru/vacancy/30592820',
            'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=30592820',
            'archived': False,
            'area': {'id': '10',
                     'name': 'Горно-Алтайск',
                     'url': 'https://api.hh.ru/areas/10'},
            'contacts': None,
            'created_at': '2019-03-16T11:37:50+0300',
            'department': None,
            'employer': {'alternate_url': 'https://hh.ru/employer/1740',
                         'id': '1740',
                         'logo_urls': {'240': 'https://hhcdn.ru/employer-logo/289170.png',
                                       '90': 'https://hhcdn.ru/employer-logo/289028.png',
                                       'original': 'https://hhcdn.ru/employer-

In [7]:
vacancy_id = request.json()['items'][0]['alternate_url'].replace('https://hh.ru/vacancy/','')
vacancy_url = 'https://api.hh.ru/vacancies/' + vacancy_id
request = requests.get(vacancy_url)
pprint(request.json())

{'accept_handicapped': False,
 'accept_incomplete_resumes': False,
 'accept_kids': False,
 'address': None,
 'allow_messages': True,
 'alternate_url': 'https://hh.ru/vacancy/30592820',
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=30592820',
 'archived': False,
 'area': {'id': '10',
          'name': 'Горно-Алтайск',
          'url': 'https://api.hh.ru/areas/10'},
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'branded_description': None,
 'code': None,
 'contacts': None,
 'created_at': '2019-03-16T11:37:50+0300',
 'department': None,
 'description': '<p>Ежедневно миллионы людей ищут различные организации. Самый '
                'простой способ найти нужную — воспользоваться сервисом '
                'Яндекс. Справочник, который является крупнейшим каталогом '
                'организаций в рунете. Операторы колл-центра следят за тем, '
                'чтобы информация в Справочнике была верной и актуальной.</p> '
                '<p> </p> <p